In [1]:
from config import drivers

In [2]:
from multiprocessing import Queue
from common import AnswerBriefInfo, AnswerParseTask, TaskStopEvent
from typing import List, Dict
from bs4 import BeautifulSoup as bf
from selenium.common.exceptions import ElementNotVisibleException
from util.web_util import zhihu_answer_url, close_button
from time import sleep

In [3]:
from selenium import webdriver

In [4]:
print(drivers["chrome"])

/media/ywatcher/ExtDisk1/Files/Packages/web_archiv/driver/chromedriver-linux64/chromedriver


In [5]:
driver = webdriver.Chrome(drivers["chrome"])

In [6]:
aid = 80059096

In [7]:
url = zhihu_answer_url(aid)
print(url)

https://www.zhihu.com/answer/80059096


In [8]:
driver.get(url)
close_button(driver)

In [12]:
content_blocks = driver.find_elements_by_xpath(
            '//div[@class="QuestionAnswer-content"]'
        )
assert len(content_blocks) == 1
content_block = content_blocks[0].find_element_by_xpath(
    '//div[@class="QuestionAnswer-content"]'  # FIXME
).get_attribute("outerHTML")


In [13]:
content_block

'<div class="QuestionAnswer-content" tabindex="0"><div><div class="ContentItem AnswerItem" data-zop="{&quot;authorName&quot;:&quot;知乎用户L6GJ5I&quot;,&quot;itemId&quot;:80059096,&quot;title&quot;:&quot;为什么做设计的都喜欢用苹果电脑？&quot;,&quot;type&quot;:&quot;answer&quot;}" name="80059096" itemprop="mainEntityOfPage" itemtype="http://schema.org/Answer" itemscope="" data-za-detail-view-path-module="AnswerItem" data-za-extra-module="{&quot;card&quot;:{&quot;has_image&quot;:false,&quot;has_video&quot;:false,&quot;content&quot;:{&quot;type&quot;:&quot;Answer&quot;,&quot;token&quot;:&quot;80059096&quot;,&quot;upvote_num&quot;:61,&quot;comment_num&quot;:24,&quot;publish_timestamp&quot;:null,&quot;parent_token&quot;:&quot;19693224&quot;,&quot;author_member_hash_id&quot;:&quot;d6e3baec6c6adf6fce7a79228de8174e&quot;}}}"><div class="ContentItem-meta"><div class="AuthorInfo AnswerItem-authorInfo AnswerItem-authorInfo--related" itemprop="author" itemscope="" itemtype="http://schema.org/Person"><div class="Autho

In [ ]:
@staticmethod
    def parse_content_block(html_text: str) -> Dict[str, str]:
        obj = bf(html_text, 'html.parser')
        
        content = obj.find(
            'div', class_='RichContent-inner').find('span').contents
        author = None  # TODO
        upvoteCount = obj.find('meta', itemprop="upvoteCount").get("content")
        dateCreated = obj.find('meta', itemprop="dateCreated").get("content")
        dateModified = obj.find('meta', itemprop="dateModified").get("content")
        commentCount = obj.find('meta', itemprop='commentCount').get("content")
        return {
            "author": author,
            "content": content,
            "upvoteCount": upvoteCount,
            "dateCreated": dateCreated,
            "dateModified": dateModified,
            "commentCount": commentCount
        }

In [14]:
obj = bf(content_block, 'html.parser')

In [17]:
author_info = obj.find('div',class_='AuthorInfo')

In [20]:
[
    author_info.find('meta',itemprop='name').get('content'),
    author_info.find('meta',itemprop='url').get('content')
]

['知乎用户L6GJ5I', 'https://www.zhihu.com/people/Nclshtl']

In [22]:
author_info.find('a',class_='UserLink-link').get('href') # real name

'//www.zhihu.com/people/Nclshtl'